# Office

In [72]:
import pandas as pd

## Database connection

In [73]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [74]:
sede = pd.read_sql_table("sede", OLTP_connection)
city = pd.read_sql_table("ciudad", OLTP_connection)

In [75]:
sede.head(1)

,sede_id,nombre,direccion,telefono,nombre_contacto,ciudad_id,cliente_id
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4


In [76]:
city.head(1)

,ciudad_id,nombre,departamento_id
0,6,BUGA,1


## Transformation

In [77]:
office_dimension = sede.merge(city, on="ciudad_id", how="left")

In [78]:
office_dimension = office_dimension.rename(
	columns={
		"nombre_x": "office_name",
		"nombre_y": "office_city",
		"departamento_id": "office_region",
	}
)

In [79]:
office_dimension.drop(
	columns=[
		"sede_id",
		"direccion",
		"telefono",
		"nombre_contacto"
	], inplace=True
)

In [80]:
office_dimension.reset_index(inplace=True)
office_dimension.rename(columns={ "index": "office_id" }, inplace=True)
office_dimension.set_index("office_id", inplace=True)

In [81]:
office_dimension.head(1)

,office_name,ciudad_id,cliente_id,office_city,office_region
office_id,,,,,
0,FARALLONES /123,1,4,CALI,1


## Load

In [82]:
office_dimension.to_sql(
	"OFFICE_DIMENSION", OLAP_connection, if_exists="replace", index=False
)

52